<a href="https://colab.research.google.com/github/yoshua-dz/POO/blob/main/Yoshua_Diaz_EuroFly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import uuid
from datetime import datetime, time

class Aerolinea:
    def __init__(self, nombre):
      self.nombre = nombre
      self.destinos = []
      self.vuelos = []
      self.horarios_validos = [time(5, 0), time(12, 0), time(22, 0)]
      self.dias_validos = ['monday', 'friday', 'saturday']

    def agregar_destino(self, destino):
      if destino not in self.destinos:
        self.destinos.append(destino)
        print("\n - - - - Nuevo destino agregado - - - - ")
      else:
        print("\n - - - - Destino ya existente - - - -")

    def eliminar_destino(self, destino):
      if destino in self.destinos:
        self.destinos.remove(destino)
        print("\n * * * * Destino eliminado * * * * ")
      else:
        print("\n * * * * Destino no encontrado * * * * ")

    def mostrar_destinos(self):
      if not self.destinos:
        print("\n - * - * - * - * No hay destinos disponibles * - * - * - * -")
      else:
        print("\nDestinos disponibles:")
        for indice, d in enumerate(self.destinos):
          print(f"{indice+1} - {d}")

    def crear_vuelo(self, destino, fecha_str, hora_str):
      if destino not in self.destinos:
        print("* * * * * Destino no disponible * * * * *")
        return
      try:
        fecha = datetime.strptime(fecha_str, "%Y-%m-%d").date() # .date() hace que solo tome los valores Y, M, D, no horas
        hora = datetime.strptime(hora_str, "%H:%M").time() # .time() hace que solo se tome los valores de horas y minutos
      except ValueError:
        print("\n* * * * *Formato de fecha u hora incorrecto* * * * * *")
        return
      if hora not in self.horarios_validos:
        print("\n* * * {} es hora no permitida. Solo se permiten 5:00, 12:00 y 22:00 * * *".format(hora))
        return
      dia_semana = fecha.strftime('%A').lower() # .strftime('%A') toma fecha y le asigna el dia en que cae, en inglés y en un string
      if dia_semana not in self.dias_validos:
        print("\n* * * {} es día no permitido. Solo lunes, viernes y sábado * * *".format(fecha.strftime('%A').lower()))
        return
      numero = str(uuid.uuid4())[0:4]
      nuevo_vuelo = Vuelo(destino, fecha, hora, numero)
      self.vuelos.append(nuevo_vuelo)
      print(f"\n- - - - Vuelo creado - - - - {nuevo_vuelo.mostrar_info()} ")

    def buscar_vuelos(self, destino):
        disponibles = [v for v in self.vuelos if v.destino.lower() == destino.lower() and v.asientos_disponibles > 0] # filtra vuelos segun nombre y si hay asientos libres aún
        if not disponibles:
            print("\n* * * No hay vuelos disponibles para ese destino * * *")
        for vuelo in disponibles:
            print(vuelo.mostrar_info())
        return disponibles

    def cancelar_vuelo(self, numero):
        for vuelo in self.vuelos:
          if vuelo.numero == numero:
            self.vuelos.remove(vuelo)
            print("\n= = = = Vuelo {} cancelado correctamente = = = =".format(numero))
            return
        print("\n- - - No se encontró el vuelo con número {} - - -".format(numero))

class Vuelo:
    def __init__(self, destino, fecha, hora, numero, capacidad=100):
      self.destino = destino
      self.fecha = fecha
      self.hora = hora
      self.numero = numero
      self.capacidad = capacidad
      self.asientos_disponibles = capacidad
      self.pasajeros = []

    def reservar(self, pasajero, cantidad):
      if cantidad <= self.asientos_disponibles:
        self.pasajeros.append(pasajero)
        self.asientos_disponibles -= cantidad
        return Reserva(pasajero, self, cantidad) # manda a llamar a Reserva, enviándose a sí mismo como argumento (self)
      else:
        print("\n* * * * * No hay suficientes asientos disponibles * * * * *")
        return None

    def cancelar_reserva(self, reserva):
      reserva.cancelar()

    def mostrar_info(self):
      return (f"""
    Vuelo {self.numero} a {self.destino}
    Fecha: {self.fecha}, Hora: {self.hora}
    Asientos disponibles: {self.asientos_disponibles} """)

class Pasajero:
    def __init__(self, nombre, pasaporte, correo):
      self.nombre = nombre
      self.pasaporte = pasaporte
      self.correo = correo

    def mostrar_informacion(self):
      print(f"""\nNombre: {self.nombre}
            Pasaporte: {self.pasaporte}
            Correo: {self.correo}""")

    @staticmethod          # No requiero inicializar un objeto para usar este método
    def pedir_info():      # por lo que no requiero un (self)
      nombre = input("Ingrese su nombre: ")
      pasaporte = input("Ingrese su pasaporte: ")
      correo = input("Ingrese su correo: ")
      return Pasajero(nombre, pasaporte, correo)

class Reserva:
    def __init__(self, pasajero, vuelo, asientos):
      self.pasajero = pasajero
      self.vuelo = vuelo
      self.asientos = asientos
      self.codigo = self.generar_codigo()

    def generar_codigo(self):
      return str(uuid.uuid4())[0:6]  #Genera un Universal Unique Identifier

    def mostrar_info(self):
      return (f"""\nCódigo: {self.codigo}
    Pasajero: {self.pasajero.nombre}
    Vuelo: {self.vuelo.numero}
    Destino: {self.vuelo.destino}
    Fecha: {self.vuelo.fecha}
    Hora: {self.vuelo.hora}
    Asientos: {self.asientos}""")

    def cancelar(self):
      self.vuelo.asientos_disponibles += self.asientos
      self.vuelo.pasajeros.remove(self.pasajero)

admin_password = "Pepe el Pollo"
reservas = []

def permiso_admin():
  contraseña = input("Ingrese la contraseña del administrador: ")
  if contraseña == admin_password:
    return True
  else:
    print("* * * Contraseña inválida * * *")
    return False

def menu_admin():
    while True:

      option = input("""\nBienvenido administrador

            Seleccione una opción
            1) Crear vuelos
            2) Eliminar vuelos
            3) Agregar destinos
            4) Eliminar destinos
            5) Salir del menu de admin
            -  """)
      if option == '1':
        destino = input("Ingrese el destino deseado: ")
        fecha_str = input("Ingrese la fecha en el formato YYYY-MM-DD \n( Lunes / Viernes / Sábado ): ")
        hora_str = input("Ingrese la hora en el formato HH:MM \n( 05:00 / 12:00 / 22:00 ) :  ")
        EuroFly.crear_vuelo(destino, fecha_str, hora_str)
      elif option == '2':
        numero = input("Imgrese el numero del vuelo: ")
        EuroFly.cancelar_vuelo(numero)
      elif option == '3':
        destino = input("Ingrese el nuevo destino: ")
        EuroFly.agregar_destino(destino)
      elif option == '4':
        destino = input("Ingrese el destino a eliminar: ")
        EuroFly.eliminar_destino(destino)
      elif option == '5':
        print("\n////////  Saliendo del menu de administrador ////////")
        break
      else:
        print("\n* * * Opción no válida, intente nuevamente * * *")

def menu(Eurofly):
   while True:
      option = input("""\nSeleccione una opción:
      1) Mostrar destinos disponibles
      2) Buscar vuelos
      3) Reservar vuelo
      4) Ver mis reservaciones
      5) Cancelar reservación
      6) Entrar como administrador
      7) Salir
      -  """)
      if option == "1":
        Eurofly.mostrar_destinos()

      elif option == "2":
        destino = input("Ingrese el destino: ")
        EuroFly.buscar_vuelos(destino)

      elif option == "3":
            pasajero = Pasajero.pedir_info()
            destino = input("Ingrese el destino deseado: ")
            vuelos = EuroFly.buscar_vuelos(destino)
            if vuelos:
                numero = input("\nIngrese el número de vuelo que desea reservar: ")
                vuelo = next((v for v in vuelos if v.numero == numero), None) #busca el vuelo, si no hay, regresa un None
                if vuelo:
                    try:
                        asientos = int(input("¿Cuántos asientos desea reservar?: "))
                        reserva = vuelo.reservar(pasajero, asientos)
                        if reserva:
                            reservas.append(reserva)
                            print("\n- - - Reserva realizada con éxito - - -")
                            print(reserva.mostrar_info())
                    except ValueError:
                        print("\n* * * * * Cantidad inválida * * * * *")
                else:
                    print("\n* * * * * Vuelo no encontrado * * * * *")

      elif option == "4":
            nombre = input("Ingrese su nombre para buscar sus reservas: ")
            encontradas = [r for r in reservas if r.pasajero.nombre.lower() == nombre.lower()]  #r.pasajero.nombre.lower() que nombre más largo por dioh
            if encontradas:
                for r in encontradas:
                    print(r.mostrar_info())
            else:
                print("\n* * * No se encontraron reservas a su nombre * * *")

      elif option =='5':
        nombre = input("Ingrese su nombre: ")
        codigo = input("Ingrese el código de la reserva a cancelar: ")
        reserva = next((r for r in reservas if r.pasajero.nombre.lower() == nombre.lower() and r.codigo == codigo), None)
        if reserva:
            reserva.cancelar()
            reservas.remove(reserva)
            print("\n- - - Reserva cancelada con éxito - - -")
        else:
            print("\n* * * Reserva no encontrada * * *")

      elif option == "6":
        if permiso_admin():
          menu_admin()

      elif option == '7':
        print("\n    Gracias por usar EuroFly. ¡Hasta pronto!")
        break

      else:
        print("\n* * * Opción inválida, intente nuevamente * * *")

EuroFly = Aerolinea("EuroFly")
EuroFly.agregar_destino("España")
EuroFly.agregar_destino("Paises Bajos")
EuroFly.agregar_destino("Suiza")
EuroFly.crear_vuelo("España", "2025-09-20", "05:00")
EuroFly.crear_vuelo("España", "2025-10-25", "22:00")
EuroFly.crear_vuelo("Paises Bajos", "2025-09-26", "05:00")
EuroFly.crear_vuelo("Paises Bajos", "2025-10-31", "12:00")
EuroFly.crear_vuelo("Suiza", "2025-09-15", "22:00")
EuroFly.crear_vuelo("Suiza", "2025-11-08", "12:00")


menu(EuroFly)



 - - - - Nuevo destino agregado - - - - 

 - - - - Nuevo destino agregado - - - - 

 - - - - Nuevo destino agregado - - - - 

- - - - Vuelo creado - - - - 
    Vuelo 6ef9 a España
    Fecha: 2025-09-20, Hora: 05:00:00
    Asientos disponibles: 100  

- - - - Vuelo creado - - - - 
    Vuelo 6575 a España
    Fecha: 2025-10-25, Hora: 22:00:00
    Asientos disponibles: 100  

- - - - Vuelo creado - - - - 
    Vuelo 3e01 a Paises Bajos
    Fecha: 2025-09-26, Hora: 05:00:00
    Asientos disponibles: 100  

- - - - Vuelo creado - - - - 
    Vuelo 8acd a Paises Bajos
    Fecha: 2025-10-31, Hora: 12:00:00
    Asientos disponibles: 100  

- - - - Vuelo creado - - - - 
    Vuelo 70e9 a Suiza
    Fecha: 2025-09-15, Hora: 22:00:00
    Asientos disponibles: 100  

- - - - Vuelo creado - - - - 
    Vuelo 43a8 a Suiza
    Fecha: 2025-11-08, Hora: 12:00:00
    Asientos disponibles: 100  

Seleccione una opción:
      1) Mostrar destinos disponibles
      2) Buscar vuelos
      3) Reservar vuelo
     